In [52]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Cargamos el DataFrame desde el archivo CSV
df = pd.read_csv('defunciones_covid19_2020_2024.csv', sep=';')

# Eliminamos las filas con región 'Ignorada'
df_filtrado = df[df['NOMBRE_REGION'] != 'Ignorada']

# Encontramos la región con el máximo número de muertes en algún año
defunciones_por_region = df_filtrado.groupby(['AÑO', 'NOMBRE_REGION']).size().unstack(fill_value=0).max()
max_defunciones = defunciones_por_region.max()

# Creamos una lista de años únicos en el DataFrame
años_unicos = df['AÑO'].unique()

# Creamos widgets para cada gráfico
año_widget_region = widgets.SelectionSlider(
    options=años_unicos,
    value=años_unicos[0],
    description='Año:',
    continuous_update=False
)

año_widget_sexo = widgets.SelectionSlider(
    options=años_unicos,
    value=años_unicos[0],
    description='Año:',
    continuous_update=False
)

año_widget_edad = widgets.SelectionSlider(
    options=años_unicos,
    value=años_unicos[0],
    description='Año:',
    continuous_update=False
)

# Función para graficar las defunciones totales por región según el año seleccionado

# Definir max_deaths_all_years
max_deaths_all_years = df[df['AÑO'] == max_deaths_year]['COMUNA'].value_counts().max()

# Definir place_mapping
place_mapping = {
    'Hospital o Clínica': 'Hospital',
    'Casa habitación': 'Casa habitación'
}
def plot_defunciones_por_region(año):
    df_año = df_filtrado[df_filtrado['AÑO'] == año]
    defunciones_por_region = df_año['NOMBRE_REGION'].value_counts().sort_values(ascending=False)

    plt.figure(figsize=(10, 6))
    defunciones_por_region.plot(kind='bar', color='skyblue')

    plt.title(f'Defunciones por región en {año}')
    plt.xlabel('Región')
    plt.ylabel('Número de defunciones')
    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, max_defunciones * 1.1)

    plt.tight_layout()
    plt.show()



# Función para graficar las defunciones por sexo según el año seleccionado
def plot_defunciones_por_sexo(año):
    df_año = df[df['AÑO'] == año]
    fallecidos_por_sexo = df_año['SEXO_NOMBRE'].value_counts()

    plt.figure(figsize=(6, 6))
    plt.pie(fallecidos_por_sexo, labels=fallecidos_por_sexo.index, autopct='%1.1f%%', colors=['lightcoral', 'lightskyblue'])
    plt.title(f'Fallecidos por sexo en {año}')
    plt.show()

# Muertes en cada año según edad
max_deaths_year = df['AÑO'].value_counts().idxmax()

df_max_deaths_year = df[df['AÑO'] == max_deaths_year]

max_deaths = df_max_deaths_year['EDAD_CANT'].value_counts().max()

def plot_defunciones_por_edad(año):
    df_year = df[df['AÑO'] == año]
    defunciones_por_edad = df_year['EDAD_CANT'].value_counts().sort_index()
    
    plt.figure(figsize=(10, 6))
    defunciones_por_edad.plot()
    plt.title(f'Distribución de defunciones por edad - Año {año}')
    plt.xlabel('Edad')
    plt.ylabel('Número de defunciones')
    plt.ylim(0, max_deaths * 1.1)
    plt.show()

# Creamos pestañas
tab = widgets.Tab()

# Creamos los gráficos para la primera pestaña
graficos_pestana_1 = widgets.VBox([
    widgets.HBox([año_widget_region, widgets.interactive_output(plot_defunciones_por_region, {'año': año_widget_region})]),
    widgets.HBox([año_widget_sexo, widgets.interactive_output(plot_defunciones_por_sexo, {'año': año_widget_sexo})]),
    widgets.HBox([año_widget_edad, widgets.interactive_output(plot_defunciones_por_edad, {'año': año_widget_edad})])
])







# Creamos los widgets y las funciones para el segundo conjunto de gráficos

# Filtramos el DataFrame para obtener solo los datos de la Región Metropolitana de Santiago
df_metropolitana = df[df['NOMBRE_REGION'] == 'Metropolitana de Santiago']

# Obtenemos los años únicos en la Región Metropolitana de Santiago
años_metropolitana = df_metropolitana['AÑO'].unique()

def plot_muertes_por_año_metropolitana(año):
    df_year = df[df['AÑO'] == año]
    df_rm = df_year[df_year['NOMBRE_REGION'] == 'Metropolitana de Santiago']
    fallecidos_por_comuna = df_rm['COMUNA'].value_counts()

    plt.figure(figsize=(10, 6))
    fallecidos_por_comuna.plot(kind='bar')
    plt.title(f'Fallecidos por COVID-19 por comuna en la Región Metropolitana - Año {año}')
    plt.xlabel('Comuna')
    plt.ylabel('Número de Fallecidos')
    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, max_deaths_all_years * 1.1)
    plt.tight_layout()
    plt.show()



# Función para graficar las defunciones por sexo en la Región Metropolitana según el año seleccionado
def plot_muertes_por_sexo_metropolitana(año):
    df_año = df_metropolitana[df_metropolitana['AÑO'] == año]
    muertes_por_sexo = df_año['SEXO_NOMBRE'].value_counts()

    plt.figure(figsize=(6, 6))
    plt.pie(muertes_por_sexo, labels=muertes_por_sexo.index, autopct='%1.1f%%', colors=['lightcoral', 'lightskyblue'])
    plt.title(f'Muertes por sexo en la Región Metropolitana de Santiago en {año}')
    plt.show()



def plot_fallecidos_por_lugar_defuncion_metropolitana(año):
    df_year = df[(df['NOMBRE_REGION'] == 'Metropolitana de Santiago') & (df['AÑO'] == año)]
    df_year.loc[:, 'LUGAR_DEFUNCION'] = df_year['LUGAR_DEFUNCION'].map(place_mapping).fillna('Otro')
    place_of_death_counts = df_year['LUGAR_DEFUNCION'].value_counts()

    plt.figure(figsize=(8, 6))
    plt.pie(place_of_death_counts, labels=place_of_death_counts.index, autopct='%1.1f%%', startangle=140)
    plt.title(f'Porcentaje de fallecidos por lugar de defunción en la Región Metropolitana en {año}')
    plt.axis('equal')
    plt.show()

# Creamos los widgets y las funciones para el segundo conjunto de gráficos
año_widget_metropolitana_2 = widgets.SelectionSlider(
    options=años_metropolitana,
    value=años_metropolitana[0],
    description='Año:',
    continuous_update=False
)

año_widget_metropolitana_3_grafico_2 = widgets.SelectionSlider(
    options=años_metropolitana,
    value=años_metropolitana[0],
    description='Año:',
    continuous_update=False
)

año_widget_metropolitana_3_grafico_3 = widgets.SelectionSlider(
    options=años_metropolitana,
    value=años_metropolitana[0],
    description='Año:',
    continuous_update=False
)

# Creamos los gráficos para la segunda pestaña
graficos_pestana_2 = widgets.VBox([
    widgets.HBox([año_widget_metropolitana_2, widgets.interactive_output(plot_muertes_por_año_metropolitana, {'año': año_widget_metropolitana_2})]),
    widgets.HBox([año_widget_metropolitana_3_grafico_2, widgets.interactive_output(plot_fallecidos_por_lugar_defuncion_metropolitana, {'año': año_widget_metropolitana_3_grafico_2})]),
    widgets.HBox([año_widget_metropolitana_3_grafico_3, widgets.interactive_output(plot_muertes_por_sexo_metropolitana, {'año': año_widget_metropolitana_3_grafico_3})])
])





#los rios


# Filtrar los datos para la región de Los Ríos
df_los_rios = df[df['NOMBRE_REGION'] == 'De Los Ríos']

# Obtener los años únicos en la Región de Los Ríos
años_los_rios = df_los_rios['AÑO'].unique()

# Encontrar la ciudad con más defunciones en algún año en Los Ríos
ciudad_con_mas_muertes = df_los_rios.groupby(['AÑO', 'COMUNA']).size().unstack().idxmax().values[0]

# Obtener el número de defunciones de esa ciudad
max_defunciones_ciudad = df_los_rios[df_los_rios['COMUNA'] == ciudad_con_mas_muertes].shape[0]

# Función para graficar las defunciones por región en la Región de Los Ríos según el año seleccionado
def plot_defunciones_por_region_los_rios(año):
    df_año = df_los_rios[df_los_rios['AÑO'] == año]
    defunciones_por_region = df_año['COMUNA'].value_counts().sort_values(ascending=False)

    plt.figure(figsize=(10, 6))
    defunciones_por_region.plot(kind='bar', color='skyblue')

    plt.title(f'Defunciones por comuna en la región de Los Ríos - Año {año}')
    plt.xlabel('Comuna')
    plt.ylabel('Número de defunciones')
    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, max_defunciones_los_rios * 1.1)

    plt.tight_layout()
    plt.show()



# Función para graficar las defunciones por edad en la Región de Los Ríos según el año seleccionado
def plot_defunciones_por_edad_los_rios(año):
    df_año = df_los_rios[df_los_rios['AÑO'] == año]
    defunciones_por_edad = df_año['EDAD_CANT'].value_counts().sort_index()
    
    plt.figure(figsize=(10, 6))
    defunciones_por_edad.plot()
    plt.title(f'Distribución de defunciones por edad en la región de Los Ríos - Año {año}')
    plt.xlabel('Edad')
    plt.ylabel('Número de defunciones')
    plt.ylim(0, max_defunciones_los_rios * 1.1)
    plt.show()
    

# Función para graficar las defunciones por sexo en la Región de Los Ríos según el año seleccionado
def plot_defunciones_por_sexo_los_rios(año):
    df_año = df_los_rios[df_los_rios['AÑO'] == año]
    fallecidos_por_sexo = df_año['SEXO_NOMBRE'].value_counts()

    plt.figure(figsize=(6, 6))
    plt.pie(fallecidos_por_sexo, labels=fallecidos_por_sexo.index, autopct='%1.1f%%', colors=['lightcoral', 'lightskyblue'])
    plt.title(f'Defunciones por sexo en la región de Los Ríos en {año}')
    plt.show()


# Creamos los widgets para Los Ríos
año_widget_los_rios = widgets.SelectionSlider(
    options=años_los_rios,
    value=años_los_rios[0],
    description='Año:',
    continuous_update=False
)

año_widget_los_rios_2 = widgets.SelectionSlider(
    options=años_los_rios,
    value=años_los_rios[0],
    description='Año:',
    continuous_update=False
)

año_widget_los_rios_3 = widgets.SelectionSlider(
    options=años_los_rios,
    value=años_los_rios[0],
    description='Año:',
    continuous_update=False
)

# Creamos los gráficos para Los Ríos en la tercera pestaña
graficos_pestana_3_los_rios = widgets.VBox([
    widgets.HBox([año_widget_los_rios, widgets.interactive_output(plot_defunciones_por_region_los_rios, {'año': año_widget_los_rios})]),
    widgets.HBox([año_widget_los_rios_2, widgets.interactive_output(plot_defunciones_por_sexo_los_rios, {'año': año_widget_los_rios_2})]),
    widgets.HBox([año_widget_los_rios_3, widgets.interactive_output(plot_defunciones_por_edad_los_rios, {'año': año_widget_los_rios_3})])
])


# Agregamos los gráficos a las pestañas
tab.children = [graficos_pestana_1, graficos_pestana_2,graficos_pestana_3]

# Nombramos las pestañas
tab.set_title(0, 'Muertes Nacional')
tab.set_title(1, 'Muertes Metropolitanas')
tab.set_title(2, 'Muertes Los Ríos')


# Mostramos el dashboard
display(tab)

